In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

/usr/local/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Here, we develop tools to pick up data needed for calibration

In [2]:
from LeCheapEyeTracker import Client, Server, Stimulation

import numpy as np
import matplotlib.pyplot as plt
import time

import cv2
from vispy import gloo, app

## First, we need to pick up stimuli x_positions.

In [3]:
class NeoStimulation(Stimulation):
    def get_stimulus(self, t0, t):
        i = int((t-t0)//self.transition_lag)
        stim_x, stim_y = self.tabPos[i]
        return self.draw_stimulus((stim_x, stim_y)), stim_x

In [4]:
class NeoClient(Client):
    def __init__(self, et, timeline, stim_type = 'calibration'):
        Client.__init__(self, et, timeline, stim_type)
        self.stimulation = NeoStimulation(self.width, self.height, stim_type = stim_type)
        self.stims_posX = np.array([])
        self.stims_time = np.array([])
        
    def on_draw(self, event):
        gloo.clear('black')
        if time.time()-self.start < self.timeline.max():
            image, x = self.stimulation.get_stimulus(t0 = self.start, t = time.time())
            self.stims_posX = np.append(self.stims_posX, x)
            self.stims_time = np.append(self.stims_time, time.time()-self.start)
            self.program['texture'][...] = image.astype(np.uint8).reshape((self.height, self.width, 3))
            self.program.draw('triangle_strip')
        else:
            self.close()

In [5]:
fps = 100
screen = NeoClient(et=Server(), timeline=np.linspace(0, 7., 3.*fps))
screen.app.run()

window size :  (2560, 1440)
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame

/usr/local/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)


In [1]:
print (screen.stims_posX)

NameError: name 'screen' is not defined

In [7]:
print (screen.stims_time)

[ 0.04832721  0.18893123  0.24525309  0.29833722  0.34892821  0.397717
  0.44847703  0.49935508  0.54888105  0.59722114  0.64776111  0.69792104
  0.74575305  0.79595923  0.84384704  0.89471102  0.94559908  0.99697208
  1.04664707  1.09458423  1.14400721  1.19219422  1.24113607  1.29004216
  1.34031701  1.39271212  1.44327116  1.49134803  1.54073811  1.58882308
  1.63816524  1.68598819  1.73526907  1.78330207  1.83330512  1.88466406
  1.93097305  1.98071909  2.02742314  2.07724619  2.12501001  2.17569208
  2.22299504  2.27259612  2.31959105  2.36979103  2.42026901  2.46813416
  2.51899314  2.56877017  2.617872    2.66729903  2.71878505  2.7689271
  2.8165791   2.86729622  2.91488218  2.96456814  3.01259613  3.06282711
  3.1128571   3.16272306  3.211797    3.26184702  3.30936718  3.35887623
  3.40629506  3.4560492   3.50359416  3.55280709  3.60138702  3.65030217
  3.69770408  3.74701118  3.79514718  3.84516621  3.89565921  3.946527
  3.99706602  4.04490113  4.09535718  4.14214301  4.1919

## Time to pick up eye x_positions

In [8]:
print (screen.et.eye_pos)

[]


Ooops

In [9]:
! git commit -am 'have to fix processing frame'

[master 7c57ff3] have to fix processing frame
 1 file changed, 77 insertions(+), 5 deletions(-)


In [10]:
! git push

Counting objects: 4, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.38 KiB | 0 bytes/s, done.
Total 4 (delta 3), reused 0 (delta 0)
To https://github.com/meduz/LeCheapEyeTracker
   38d8acc..7c57ff3  master -> master
